# data-attest-ezkl

Here's an example leveraging EZKL whereby the inputs to the model are read and attested to from an on-chain source.

In this setup:
- the inputs and outputs are publicly known to the prover and verifier
- the on chain inputs will be fetched and then fed directly into the circuit
- the quantization of the on-chain inputs happens within the evm and is replicated at proving time 


First we import the necessary dependencies and set up logging to be as informative as possible. 

In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


from torch import nn
import ezkl
import os
import json
import logging

# uncomment for more descriptive logging 
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)


Now we define our model. It is a very simple PyTorch model that has just one layer, an average pooling 2D layer. 

In [2]:
import torch
# Defines the model

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer = nn.AvgPool2d(2, 1, (1, 1))

    def forward(self, x):
        return self.layer(x)[0]


circuit = MyModel()

# this is where you'd train your model

We omit training for purposes of this demonstration. We've marked where training would happen in the cell above. 
Now we export the model to onnx and create a corresponding (randomly generated) input. This input data will eventually be stored on chain and read from according to the call_data field in the graph input.

You can replace the random `x` with real data if you so wish. 

In [3]:
x = 0.1*torch.rand(1,*[3, 2, 2], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      "network.onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump(data, open("input.json", 'w' ))



We now define a function that will create a new anvil instance which we will deploy our test contract too. This contract will contain in its storage the data that we will read from and attest to. In production you would not need to set up a local anvil instance. Instead you would replace RPC_URL with the actual RPC endpoint of the chain you are deploying your verifiers too, reading from the data on said chain.

In [4]:
import subprocess
import time
import threading

# make sure anvil is running locally
# $ anvil -p 3030

RPC_URL = "http://localhost:3030"

# Save process globally
anvil_process = None

def start_anvil():
    global anvil_process
    if anvil_process is None:
        anvil_process = subprocess.Popen(["anvil", "-p", "3030", "--code-size-limit=41943040"])
        if anvil_process.returncode is not None:
            raise Exception("failed to start anvil process")
        time.sleep(3)

def stop_anvil():
    global anvil_process
    if anvil_process is not None:
        anvil_process.terminate()
        anvil_process = None


We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

Here we create the following setup:
- `input_visibility`: "public"
- `param_visibility`: "private"
- `output_visibility`: public


In [5]:
import ezkl

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
data_path = os.path.join('input.json')

run_args = ezkl.PyRunArgs()
run_args.input_visibility = "public"
run_args.param_visibility = "private"
run_args.output_visibility = "public"
run_args.num_inner_cols = 1
run_args.variables = [("batch_size", 1)]





Now we generate a settings file. This file basically instantiates a bunch of parameters that determine their circuit shape, size etc... Because of the way we represent nonlinearities in the circuit (using Halo2's [lookup tables](https://zcash.github.io/halo2/design/proving-system/lookup.html)), it is often best to _calibrate_ this settings file as some data can fall out of range of these lookups.

You can pass a dataset for calibration that will be representative of real inputs you might find if and when you deploy the prover. Here we create a dummy calibration dataset for demonstration purposes. 

In [6]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True

DEBUG tract_onnx.model 2024-05-10 02:37:36,565 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-05-10 02:37:36,566 model.rs:643 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,567 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,568 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,569 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,570 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,571 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0 } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,571 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,572 change_axes.rs:11

In [7]:
# generate a bunch of dummy calibration data
cal_data = {
    "input_data": [(0.1*torch.rand(2, *[3, 2, 2])).flatten().tolist()],
}

cal_path = os.path.join('val_data.json')
# save as json file
with open(cal_path, "w") as f:
    json.dump(cal_data, f)

res = ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

DEBUG tract_onnx.model 2024-05-10 02:37:36,598 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-05-10 02:37:36,598 model.rs:643 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,599 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,599 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:36,599 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,599 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,600 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0 } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,600 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:36,600 change_axes.rs:11

In [8]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

DEBUG tract_onnx.model 2024-05-10 02:37:38,796 model.rs:254 ONNX operator set version: 10
DEBUG ezkl.graph.model 2024-05-10 02:37:38,796 model.rs:643 set batch_size to 1
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:38,797 analyser.rs:151   Refined 1/0>: ..,? -> batch_size,3,3,3,F32
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:38,797 analyser.rs:151   Refined 2/0>: ..,? -> ,I64 0
DEBUG tract_hir.infer.analyser 2024-05-10 02:37:38,797 analyser.rs:151   Refined 3/0>: ..,? -> 3,3,3,F32
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:38,798 change_axes.rs:76   Considering change AxisChange { outlet: 0/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:38,798 change_axes.rs:88     Change AxisChange { outlet: 0/0>, op: Rm(0 } blocked by locked interface 0/0>
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:38,798 change_axes.rs:76   Considering change AxisChange { outlet: 1/0>, op: Rm(0 }
DEBUG tract_core.optim.change_axes 2024-05-10 02:37:38,798 change_axes.rs:11

The graph input for on chain data sources is formatted completely differently compared to file based data sources.

- For file data sources, the raw floating point values that eventually get quantized, converted into field elements and stored in `witness.json` to be consumed by the circuit are stored. The output data contains the expected floating point values returned as outputs from running your vanilla pytorch model on the given inputs.
- For on chain data sources, the input_data field contains all the data necessary to read and format the on chain data into something digestable by EZKL (aka field elements :-D). 
Here is what the schema for an on-chain data source graph input file should look like:
    
```json
{
  "input_data": {
    "rpc": "http://localhost:3030", // The rpc endpoint of the chain you are deploying your verifier to
    "calls": [
      {
        "call_data": [
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000000", // The abi encoded call data to a view function that returns a single on-chain data point (we only support uint256 returns for now)
            7 // The number of decimal places of the large uint256 value. This is our way of representing large wei values as floating points on chain, since the evm only natively supports integer values.
          ],
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000001",
            5
          ],
          [
            "71e5ee5f0000000000000000000000000000000000000000000000000000000000000002",
            5
          ]
        ],
        "address": "5fbdb2315678afecb367f032d93f642f64180aa3" // The address of the contract that we are calling to get the data. 
      }
    ]
  }
}

In [9]:
await ezkl.setup_test_evm_witness(
    data_path,
    compiled_model_path,
    # we write the call data to the same file as the input data
    data_path,
    input_source=ezkl.PyTestDataSource.OnChain,
    output_source=ezkl.PyTestDataSource.File,
    rpc_url=RPC_URL)

DEBUG alloy_rpc_client.call 2024-05-10 02:37:38,805 call.rs:85 sending request method=eth_chainId id=0
DEBUG alloy_transport_http.reqwest 2024-05-10 02:37:38,805 reqwest.rs:18 ReqwestTransport; url=http://localhost:3030/
DEBUG reqwest.connect 2024-05-10 02:37:38,806 connect.rs:497 starting new connection: http://localhost:3030/
DEBUG hyper_util.client.legacy.connect.dns 2024-05-10 02:37:38,807 dns.rs:122 resolving host="localhost"
DEBUG hyper_util.client.legacy.connect.http 2024-05-10 02:37:38,808 http.rs:622 connecting to [::1]:3030
DEBUG hyper_util.client.legacy.connect.http 2024-05-10 02:37:38,809 http.rs:622 connecting to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.connect.http 2024-05-10 02:37:38,810 http.rs:625 connected to 127.0.0.1:3030
DEBUG hyper_util.client.legacy.pool 2024-05-10 02:37:38,811 pool.rs:397 pooling idle connection for ("http", localhost:3030)
DEBUG alloy_transport_http.reqwest 2024-05-10 02:37:38,812 reqwest.rs:30 received response from server status=200 OK
D

True

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [10]:
res = await ezkl.get_srs( settings_path)


INFO ezkl.execute 2024-05-10 02:37:38,942 execute.rs:599 SRS already exists at that path
INFO ezkl.execute 2024-05-10 02:37:38,943 execute.rs:497 read 33028 bytes from file (vector of len = 33028)
INFO ezkl.execute 2024-05-10 02:37:38,943 execute.rs:504 file hash: 446092fd1d6030e5bb2f2a8368267d5ed0fbdb6a766f6c5e4a4841827ad3106f


We now need to generate the circuit witness. These are the model outputs (and any hashes) that are generated when feeding the previously generated `input.json` through the circuit / model. 

In [11]:
!export RUST_BACKTRACE=1

witness_path = "witness.json"

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)

DEBUG ezkl.graph 2024-05-10 02:37:39,068 mod.rs:968 input scales: [13]
DEBUG alloy_rpc_client.call 2024-05-10 02:37:39,068 call.rs:85 sending request method=eth_chainId id=0
DEBUG alloy_transport_http.reqwest 2024-05-10 02:37:39,069 reqwest.rs:18 ReqwestTransport; url=http://localhost:3030/


Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [ ]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

Now we generate a full proof. 

In [ ]:
# GENERATE A PROOF

proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

And verify it as a sanity check. 

In [ ]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

We can now create and then deploy a vanilla evm verifier.

In [ ]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = await ezkl.create_evm_verifier(
        vk_path,
        
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True

In [ ]:
import json

addr_path_verifier = "addr_verifier.txt"

res = await ezkl.deploy_evm(
    addr_path_verifier,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

With the vanilla verifier deployed, we can now create the data attestation contract, which will read in the instances from the calldata to the verifier, attest to them, call the verifier and then return the result. 

In [ ]:

abi_path = 'test.abi'
sol_code_path = 'test.sol'
input_path = 'input.json'

res = await ezkl.create_evm_data_attestation(
        input_path,
        settings_path,
        sol_code_path,
        abi_path,
    )

Now we can deploy the data attest verifier contract. For security reasons, this binding will only deploy to a local anvil instance, using accounts generated by anvil. 
So should only be used for testing purposes.

In [ ]:
addr_path_da = "addr_da.txt"

res = await ezkl.deploy_da_evm(
        addr_path_da,
        input_path,
        settings_path,
        sol_code_path,
        RPC_URL,
    )


Call the view only verify method on the contract to verify the proof. Since it is a view function this is safe to use in production since you don't have to pass your private key.

In [ ]:
# read the verifier address
addr_verifier = None
with open(addr_path_verifier, 'r') as f:
    addr = f.read()
#read the data attestation address
addr_da = None
with open(addr_path_da, 'r') as f:
    addr_da = f.read()

res = await ezkl.verify_evm(
    addr,
    proof_path,
    RPC_URL,
    addr_da,
)